<a href="https://colab.research.google.com/github/HebahAlshamlan/AI/blob/master/Cyberbullying_Detection_CNN_%5BALANOUD%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cyberbullying Detection - CNN

In this notebook we are going to implement a CNN model to detects cyberbullying in Arabic social media content.
* The CNN model architecture is inspired by XXXX work.
* The model settings is specified based on XXXX.
* We will use two different word embeddings: 
    * 1- Our word embeddings trained using Gemsim library
    * 2- Aravec word embeddigs 


In [0]:
import pandas as pd
import matplotlib as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.utils import shuffle

pd.set_option('max_colwidth',400)

---

## Loading Dataset

This section of the notebook will modify the pre-processed comments' size and save the new dataset of size =11 (or any size). 

In [2]:
#This file has the comments only with empty lines
pd.set_option('max_colwidth',400)
# dataset = pd.read_excel('~/Dropbox/GP/GP - Deep Learning/Dataset/preprocessed_all_videos_word_embeddings_dataset_aravec_new.xlsx')
#data = pd.read_csv('~/Dropbox/GP/GP - Deep Learning/Dataset/preprocessed_tokenized_3_videos_nan.csv')
dataset = pd.read_excel('preprocessed_all_videos_word_embeddings_dataset_aravec_new.xlsx')

print(dataset.shape)
#print(data.shape)

dataset.head()

(8083, 705)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 665,Unnamed: 666,Unnamed: 667,Unnamed: 668,Unnamed: 669,Unnamed: 670,Unnamed: 671,Unnamed: 672,Unnamed: 673,Unnamed: 674,Unnamed: 675,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685,Unnamed: 686,Unnamed: 687,Unnamed: 688,Unnamed: 689,Unnamed: 690,Unnamed: 691,Unnamed: 692,Unnamed: 693,Unnamed: 694,Unnamed: 695,Unnamed: 696,Unnamed: 697,Unnamed: 698,Unnamed: 699,Unnamed: 700,Unnamed: 701,Unnamed: 702,Unnamed: 703,Unnamed: 704
0,1,جمع,من,ابو,اله,يكف,سبد,دبل,لا,وتن,وقت,وله,فقع,جرد,وقت,ذا,مشاهير,انت,درس,نبه,ثان,شي,اول,نحف,بس,نفس,علي,ركز,حب,كثر,شكل,نبي,ما,لان,مجاديف,كسر,لك,نسل,يعن,تبي,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,جفن,ابو,هو,سعد,ولا,كوت,هذا,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,لمع,سوي,رضو,كره,ذيع,وذا,بزر,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,لمع,سوي,رضو,كره,ذيع,وذا,بزر,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,نتم,يشر,لسب,ليش,طيب,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
dataset_labels = dataset['Unnamed: 0'].copy()
dataset_comments = dataset.copy()
dataset_comments = dataset_comments.drop(['Unnamed: 0'],axis=1)
print(dataset_labels.shape)
print(dataset_comments.shape)

(8083,)
(8083, 704)


In [4]:
print('Number of reviews before removing outliers: ', dataset_comments.shape[0])

## remove any comment/labels with zero length from the dataset.
# get indices of any reviews with length 0
zero_idx = [ii for ii in range(dataset_comments.shape[0]) if str(dataset_comments.iloc[ii][0]) == 'nan']

# remove 0-length reviews and their labels
dataset = dataset.drop(zero_idx)
dataset_comments = dataset_comments.drop(zero_idx)
dataset_labels = dataset_labels.drop(zero_idx)

print('Number of reviews after removing outliers: ', dataset_comments.shape[0])

Number of reviews before removing outliers:  8083
Number of reviews after removing outliers:  8005


In [0]:
dataset_labels.to_excel('temp.xlsx')

In [0]:
data = pd.read_excel('temp.xlsx')

In [10]:
print(dataset.shape)
print(dataset_labels.shape)
print(dataset_comments.shape)
print(data.shape)

(8005, 705)
(8005,)
(8005, 704)
(8005, 1)


In [11]:
data.rename(columns={'Unnamed: 0': 'label'}, inplace=True)
data.head()

,label
0,1
1,0
2,1
3,1
4,0


In [0]:
comments_list = []
comments_list2 = []
for i in range(dataset_comments.shape[0]):
    comment = dataset_comments.iloc[i].dropna()
    comment = comment.tolist()
    comments_list.append(comment)
    
for i in range(dataset_comments.shape[0]):
    comment = comments_list[i]
    comment = unify_dataset_size(comment, 20)
    comments_list2.append(comment)

In [17]:
comments_df = pd.DataFrame(comments_list2)
comments_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,كسر,لك,نسل,يعن,تبي,وش,هدم,نقد,صحب,يقل,رت,فقع,دقيق,كمل,قدر,ما,ياتسبديه,سمج,ذا,وش
1,هذا,كوت,ولا,سعد,هو,ابو,جفن,0,0,0,0,0,0,0,0,0,0,0,0,0
2,بزر,وذا,ذيع,كره,رضو,سوي,لمع,0,0,0,0,0,0,0,0,0,0,0,0,0
3,بزر,وذا,ذيع,كره,رضو,سوي,لمع,0,0,0,0,0,0,0,0,0,0,0,0,0
4,طيب,ليش,لسب,يشر,نتم,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
data = pd.concat([comments_df, data], axis=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,label
0,كسر,لك,نسل,يعن,تبي,وش,هدم,نقد,صحب,يقل,رت,فقع,دقيق,كمل,قدر,ما,ياتسبديه,سمج,ذا,وش,1
1,هذا,كوت,ولا,سعد,هو,ابو,جفن,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,بزر,وذا,ذيع,كره,رضو,سوي,لمع,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,بزر,وذا,ذيع,كره,رضو,سوي,لمع,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,طيب,ليش,لسب,يشر,نتم,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
data.to_excel('all_videos_dataset_cnn_20.xlsx')

## Numbers

In [0]:
comment_len = []
for i in range(dataset_comments.shape[0]):
    comment_len.append(len(dataset_comments.iloc[i].dropna()))

In [21]:
comment_len.sort()
comments_median = comment_len[len(comment_len)//2]
comments_avg = sum(comment_len)//len(comment_len)

print('\nMedian ',comments_median)
print('\nMean ',comments_avg)


Median  6

Mean  11


## Dataset size modification

This section of the code unify the dataset size, i.e. it pad short comments with zeros, and truncate long comments. 

In [0]:
def unify_dataset_size(comment, size):
    
    if len(comment)<size:
        return pad(comment,size)
    elif len(comment)>size:
        return truncate(comment,size)
        
    return comment

In [0]:
def pad(text, size):
    
    comment = []
    n = size - len(text)
    zeros = [0]*n
    i = len(text)-1
    while i>=0:
        comment.append(text[i])
        i -=1
        
    for item in zeros:
        comment.append(item) 
    
    return comment

In [0]:
def truncate(text, size):
    n = len(text)-size
    comment = text[n:]
    return comment

## Save dataset 

In [0]:
# import xlsxwriter module 
import xlsxwriter 
  
workbook = xlsxwriter.Workbook('dataset_cnn_11.xlsx') 
  
# By default worksheet names in the spreadsheet will be  
# Sheet1, Sheet2 etc., but we can also specify a name. 
worksheet = workbook.add_worksheet("My sheet") 

  
# Start from the first cell. Rows and 
# columns are zero indexed. 
row = 0
col = 0
  
# Iterate over the data and write it out row by row. 
for i in range(dataset.shape[0]): 
    lis = dataset.iloc[i][0]
    col = len(lis)
    label = col+1
    for word in lis:
        worksheet.write(row, col, word) 
        col -=1
        
    lab = dataset.iloc[i][1]
    if i == 7935:
        lab = 0
    worksheet.write(row, label, lab) 
    row += 1
    col = 0
    
workbook.close() 

---

## Loading Word Embedding model

In this section, we will impored the word embedding model we trained, and we will change the data representation to a matrix like representation. In the matrix like representation, each comment is represented by a matrix of size 11x11, each row is a word, and each word is converted to its word embeddings representation. 
* The this is saved to bb file 

In [0]:
import gensim
from gensim.models import Word2Vec
from gensim import corpora
from pprint import pprint
from gensim.utils import simple_preprocess
from smart_open import smart_open
import os
from nltk import ngrams 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [23]:
# embedding_model = Word2Vec.load("~/Dropbox/GP/GP - Deep Learning/Word Embeddings/Models/word2vec_sg_30_all_new.model")
embedding_model = Word2Vec.load("word2vec_sg_30_all_new.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [24]:
# dataset_11 = pd.read_excel('~/Dropbox/GP/GP - Deep Learning/Dataset/all_videos_dataset_cnn_20.xlsx')
dataset_11 = pd.read_excel('all_videos_dataset_cnn_20.xlsx')
dataset_11.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,label
0,0,كسر,لك,نسل,يعن,تبي,وش,هدم,نقد,صحب,يقل,رت,فقع,دقيق,كمل,قدر,ما,ياتسبديه,سمج,ذا,وش,1
1,1,هذا,كوت,ولا,سعد,هو,ابو,جفن,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,بزر,وذا,ذيع,كره,رضو,سوي,لمع,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,بزر,وذا,ذيع,كره,رضو,سوي,لمع,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,طيب,ليش,لسب,يشر,نتم,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
dataset_11 = shuffle(dataset_11)
dataset_11.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,label
3935,3935,ذي,مثل,طلع,بنت,شرد,بنت,يوم,كل,سعد,في,حصل,غرب,شي,في,فضي,كلم,قبل,عن,بعد,يخا,0
1188,1188,مو,نفع,ترا,غني,ابد,رقص,مع,رجل,اقر,قرن,بدل,اغن,0,0,0,0,0,0,0,0,0
6575,6575,بله,مو,حرم,ريم,سعد,طلق,وهذ,تزج,0,0,0,0,0,0,0,0,0,0,0,0,1
4595,4595,الي,حف,علي,اخت,يعط,ليك,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
613,613,سهل,لا,تكن,علم,من,علم,يوم,قيم,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
dataset_11_label = dataset_11['label']
dataset_11_label = dataset_11_label.to_numpy()
print(dataset_11_label.shape)

(8005,)


In [0]:
dataset_11_comment = dataset_11['Unnamed: 0'].copy() 
dataset_11 = dataset_11.drop(['Unnamed: 0', 'label'], axis=1)

In [0]:
for i in range(dataset_11.shape[0]):
    comment = dataset_11.iloc[i].dropna()
    comment = comment.tolist()
    dataset_11_comment.iloc[i] = comment

In [29]:
dataset_11_comment = pd.DataFrame(dataset_11_comment)
dataset_11_comment.head()

,Unnamed: 0
3935,"[ذي, مثل, طلع, بنت, شرد, بنت, يوم, كل, سعد, في, حصل, غرب, شي, في, فضي, كلم, قبل, عن, بعد, يخا]"
1188,"[مو, نفع, ترا, غني, ابد, رقص, مع, رجل, اقر, قرن, بدل, اغن, 0, 0, 0, 0, 0, 0, 0, 0]"
6575,"[بله, مو, حرم, ريم, سعد, طلق, وهذ, تزج, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4595,"[الي, حف, علي, اخت, يعط, ليك, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
613,"[سهل, لا, تكن, علم, من, علم, يوم, قيم, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [30]:
print(dataset_11_comment.shape)
print(dataset_11_label.shape)

(8005, 1)
(8005,)


In [31]:

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
 
train_x, remaining_x, train_y, remaining_y = train_test_split(dataset_11_comment, dataset_11_label, 
                                                    test_size=0.20, random_state=42, shuffle =False)

test_idx = int(remaining_x.shape[0]*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

train_y = np.asarray(train_y)
val_y = np.asarray(val_y)
test_y = np.asarray(test_y)

print(train_y.shape)
print(val_y.shape)
print(test_y.shape)

			Feature Shapes:
Train set: 		(6404, 1) 
Validation set: 	(800, 1) 
Test set: 		(801, 1)
(6404,)
(800,)
(801,)


In [44]:
X_train.iloc[0][0]

NameError: ignored

## Create image like comments representaion

Here we will change the comments representation from list of words to a matrix representation. Each row of the matrix is a word, and it is represented using word embeddings with dim = 150 

In [0]:
embed_vocab = []
for x in embedding_model.wv.vocab:
    embed_vocab.append(x)

In [0]:
import difflib
import random

def get_most_sim_word(word):
    res = difflib.get_close_matches(word,embed_vocab,n=1)
    if (len(res)==0):
        idx = random.randrange(len(embed_vocab))
        res = embed_vocab[idx]
    return res

In [0]:
def sen_to_matrix(sen, dim):
    
    matrix = np.empty([len(sen), dim])
    i = 0    
    for word in sen:
        if word == 0:
            rep = [0] * dim
        else:
            try:
                rep = embedding_model[word]
            except KeyError:
                sim_word = get_most_sim_word(word)
                rep = embedding_model[sim_word]
                if(len(rep)==1):
                    rep = rep[0]
        
        j=0        
        for d in rep:
            matrix[i][j] = d
            j += 1 
        i += 1
    
    return matrix

In [36]:
X_train_np = np.empty([train_x.shape[0]*20, 30])

for i in range(train_x.shape[0]):
    index = i*20
    com = train_x.iloc[i][0]
    mat = sen_to_matrix(com, 30)
    X_train_np[index:index+20] = pd.DataFrame(mat)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [37]:
X_val_np = np.empty([val_x.shape[0]*20, 30])

for i in range(val_x.shape[0]):
    index = i*20
    com = val_x.iloc[i][0]
    mat = sen_to_matrix(com, 30)
    X_val_np[index:index+20] = pd.DataFrame(mat)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [38]:
X_test_np = np.empty([test_x.shape[0]*20, 30])

for i in range(test_x.shape[0]):
    index = i*20
    com = test_x.iloc[i][0]
    mat = sen_to_matrix(com, 30)
    X_test_np[index:index+20] = pd.DataFrame(mat)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [39]:
print(X_train_np.shape[0]/20)
print(X_test_np.shape[0]/20)
print(X_train.shape[0])
print(X_test.shape[0])

6404.0
801.0


NameError: ignored

In [0]:
y_train = np.array(train_y)
y_val = np.array(val_y)
y_test = np.array(test_y)

### Save

In [0]:
from torch.utils.data import Dataset, DataLoader

class CommentsDatasetTrain(Dataset):
    def __init__(self):
        self.len = X_train_np.shape[0]//20
        self.x_data = torch.from_numpy(X_train_np)
        self.y_data = torch.from_numpy(y_train)
        
    def __getitem__(self, index):
        return self.x_data[index:index+20], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
class CommentsDatasetVal(Dataset):
    def __init__(self):
        self.len = X_val_np.shape[0]//20
        self.x_data = torch.from_numpy(X_val_np)
        self.y_data = torch.from_numpy(y_val)
        
    def __getitem__(self, index):
        return self.x_data[index:index+20], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
class CommentsDatasetTest(Dataset):
    def __init__(self):
        self.len = X_test_np.shape[0]//20
        self.x_data = torch.from_numpy(X_test_np)
        self.y_data = torch.from_numpy(y_test)
            
    def __getitem__(self, index):
        return self.x_data[index:index+20], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
batch_size = 64

In [0]:
dataset_v1_train = CommentsDatasetTrain()

train_loader = DataLoader(dataset=dataset_v1_train,
                         batch_size= batch_size,
                         num_workers=0,
                          drop_last=True)

In [0]:
dataset_v1_val = CommentsDatasetVal()

val_loader = DataLoader(dataset=dataset_v1_val,
                         batch_size=batch_size,
                         num_workers=0,
                        drop_last=True)

In [0]:
dataset_v1_test = CommentsDatasetTest()

test_loader = DataLoader(dataset=dataset_v1_test,
                         batch_size=batch_size,
                         num_workers=0,
                         drop_last=True)

In [49]:
i = 0
for data, target in train_loader:
    i += 1
    print(data.shape)
    print(target.shape)

torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Siz

In [50]:
i = 0
for data, target in test_loader:
    i += 1
    print(data.shape)
    print(target.shape)
print(i)

torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
12


In [51]:
i = 0
for data, target in val_loader:
    i += 1
    print(data.shape)
    print(target.shape)
print(i)

torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
torch.Size([64, 20, 30])
torch.Size([64])
12


## CNN model

In [1]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # convolutional layer (sees 20x30x1 image tensor)
        self.conv1 = nn.Conv2d(32, 32, 3, padding=1)
        # convolutional layer (sees 10x15x32 tensor)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        
        # convolutional layer (sees 8x8x32 tensor)
        #self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        
        # max pooling layer
        self.pool = nn.AvgPool2d(2, 2)
        
        # linear layer (5 * 7 * 32 -> 8000)
        self.fc1 = nn.Linear( 1120 , 256)
        
        # linear layer (50 -> 10)
        self.fc2 = nn.Linear(256, 1)
        # dropout layer (p=0.25)
        
        self.dropout = nn.Dropout(0.20)

    def forward(self, x):
        
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = self.dropout(x)
        # x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x_s = x.view(x.shape[0],-1)

        # add dropout layer
        #x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x_s = F.relu(self.fc1(x_s))
        # add dropout layer
        x_s = self.dropout(x_s)
        # add 2nd hidden layer, with relu activation function
        #x = self.fc1(x)
        out = self.fc2(x_s)
        return F.sigmoid(out)

# create a complete CNN
model = CNN()
print(model)

CNN(
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=1120, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [77]:
model.parameters()

<generator object Module.parameters at 0x7f211b7aef68>

In [0]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.BCELoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [0]:
# number of epochs to train the model
def training(model,e):
    n_epochs = e
    valid_losses = []
    train_losses = []
    valid_loss_min = np.Inf # track change in validation loss

    for epoch in range(1, n_epochs+1):

#         if epoch > 0 and epoch % (2) == 0:
#         #if epoch > 0:
#             print("epoch = %6d" % epoch, end="")
#             print("  batch loss = %7.4f" % loss.item(), end="")
#             acc = akkuracy(model, data, target)
#             print("  accuracy = %0.2f%%" % acc)
        
    # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
        j =0
        model.train()
        for data, target in train_loader:
        # move tensors to GPU if CUDA is available
#         if train_on_gpu:
#             data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables

            data = torch.tensor(data).type('torch.FloatTensor')
            target = torch.tensor(target).type('torch.FloatTensor')
            optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        
#         print('batch ',j)
#         #try:
#         output = [0] * 32
#         for i in range(32):
#             print(i)
#             output[i] = model(data[i].view(1,1,11,150))

#         except RuntimeError:
#             outout = [0] * 9
#             for i in range(9):
#                 outout[i] = model(data[i].view(1,1,11,150))
            
            
            output = model(data.view(batch_size,1,20,30))
        
            
        # calculate the batch loss

#             t = []
#             for i in range(target.shape[0]):
#                 if target[i] == 1:
#                     t.append([0,1])
#                 else:
#                     t.append([1,0])
#             t = torch.FloatTensor(t)
        
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
        model.eval()
        for data, target in val_loader:
        # move tensors to GPU if CUDA is available
#         if train_on_gpu:
#             data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
            data = torch.tensor(data).type('torch.FloatTensor')
            target = torch.tensor(target).type('torch.FloatTensor')
        
            output = model(data.view(batch_size,1,20,30))
        
            
        # calculate the batch loss

#             t = []
#             for i in range(target.shape[0]):
#                 if target[i] == 1:
#                     t.append([0,1])
#                 else:
#                     t.append([1,0])
#             t = torch.FloatTensor(t)
        
            loss = criterion(output, target)

            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
        train_loss = train_loss/len(train_loader.sampler)
        valid_loss = valid_loss/len(val_loader.sampler)
    
        if epoch > 0 and epoch % (n_epochs/10) == 0:
            # print training/validation statistics 
            print('Epoch: {} \tTraining Loss: {} \tValidation Loss: {}'.format(
                 epoch, train_loss, valid_loss))
            train_losses.append(train_loss)
            valid_losses.append(valid_loss)
    
    # save model if validation loss has decreased
#     if valid_loss <= valid_loss_min:
#         print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
#         valid_loss_min,
#         valid_loss))
#         torch.save(model.state_dict(), 'model_cifar.pt')
#         valid_loss_min = valid_loss

In [0]:
def testing(model):
    outputs_list = []
    labels_list = []
    
    model.eval()
    
    for data, target in test_loader:

        data = torch.tensor(data).type('torch.FloatTensor')
        target = torch.tensor(target).type('torch.FloatTensor')
        
        b = data.shape[0]
        pred_y = model(data.view(b,1,20,30))
            
#         pred_y = []
#         for i in range(batch_size):
#             if output[i][0]>0.5:
#                 pred_y.append(0)
#             else:
#                 pred_y.append(1)
        for i in range(batch_size):
            outputs_list.append(pred_y[i].item())
            labels_list.append(target[i].item())
        
    return outputs_list, labels_list        

In [58]:
j=0
for i in range(len(outputs_list)):
    if outputs_list[i]>=0.5:
        j+=1
print()

NameError: ignored

In [0]:
def akkuracy(model, data_x, data_y):
    # data_x and data_y are numpy array-of-arrays matrices
    X = torch.Tensor(data_x)
    Y = torch.Tensor(data_y)   # a Tensor of 0s and 1s
    b = data_x.shape[0]
    oupt = model(X.view(batch_size,1,20,30))            # a Tensor of floats
    pred_y = []
    for i in range(batch_size):
        if oupt[i]>0.5:
            pred_y.append(1)
        else:
            pred_y.append(0)
   
            
    # pred_y = oupt >= 0.5       # a Tensor of 0s and 1s
    pred_y = torch.Tensor(pred_y)
    print(pred_y.shape)
    num_correct = torch.sum(Y==pred_y)  # a Tensor
    
    target_names= ['Not Cyberbullying', 'Cyberbullying']

    print(confusion_matrix (pred_y, data_y))
    print(classification_report(pred_y, data_y, target_names=target_names))
    print("Accuracy: ",accuracy_score(pred_y, data_y))
    
    acc = (num_correct.item() * 100.0 / len(data_y))  # scalar
    return acc

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix 
import random


def metrics(outputs_list_n, labels_list_n):
    
    y_pred = []
    y_true = labels_list_n

    for i in range(len(outputs_list_n)):
        if(outputs_list_n[i]>=0.5):
            y_pred.append(1)
        else:
            y_pred.append(0)


    target_names= ['Not Cyberbullying', 'Cyberbullying']
    print(confusion_matrix (y_pred, y_true))
    print(classification_report(y_pred, y_true, target_names=target_names))
    print("Accuracy: ",accuracy_score(y_pred, y_true))


In [2]:
e =50
highest_acc = 0

for i in range(20):
    print("iter: ", i+1)
    training(model,e)
    outputs_list, labels_list = testing(model)

    print("------------------------------")
    print(metrics(outputs_list,labels_list))
    print("------------------------------")


iter:  1


NameError: ignored

In [0]:
y_pred = []
for j in range(len(outputs_list)):
        if(outputs_list[i]>=0.5):
            y_pred.append(1)
            print('1')
        else:
            y_pred.append(0)

In [68]:
outputs_list[59]

1.2426064131432213e-05

In [69]:
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [70]:
labels_list[0][0].item()

TypeError: ignored

In [71]:
import matplotlib.pyplot as plt

epo = np.arange(0, 2000, 200)
# red dashes, blue squares and green triangles
plt.plot(epo, train_losses)
plt.plot(epo,valid_losses)

plt.show()

# plt.xlabel('Epoch')
# plt.ylabel('Loss')

NameError: ignored

In [0]:
print(output)
print(t)

In [0]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode
'''

dtype = torch.FloatTensor

# Text-CNN Parameter
embedding_size = 150 # n-gram
sequence_length = 11
num_classes = 2  # 0 or 1
filter_sizes = [2, 2, 2] # n-gram window
num_filters = 3

# 3 words sentences (=sequence_length is 3)
# sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
# labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

# word_list = " ".join(sentences).split()
# word_list = list(set(word_list))
# word_dict = {w: i for i, w in enumerate(word_list)}
# vocab_size = len(word_dict)

# inputs = []
# for sen in sentences:
#    inputs.append(np.asarray([word_dict[n] for n in sen.split()]))

# targets = []
# for out in labels:
#    targets.append(out) # To using Torch Softmax Loss function

input_batch = Variable(torch.LongTensor(dataset_11_comment_np))
target_batch = Variable(torch.LongTensor(dataset_11_label))


class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()

        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(dtype)
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(dtype)
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)

    def forward(self, X):
        embedded_chars = self.W[X] # [batch_size, sequence_length, sequence_length]
        embedded_chars = embedded_chars.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]

        pooled_outputs = []
        for filter_size in filter_sizes:
            # conv : [input_channel(=1), output_channel(=3), (filter_height, filter_width), bias_option]
            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias=True)(embedded_chars)
            h = F.relu(conv)
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length - filter_size + 1, 1))
            # pooled : [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3)]
            pooled = mp(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)

        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) # [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3) * 3]
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel * 3)]

        model = torch.mm(h_pool_flat, self.Weight) + self.Bias # [batch_size, num_classes]
        return model

model = TextCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(50):
    optimizer.zero_grad()
    output = model(input_batch)

    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

# Test
# test_text = 'sorry hate you'
# tests = [np.asarray([word_dict[n] for n in test_text.split()])]
# test_batch = Variable(torch.LongTensor(tests))

# # Predict
# predict = model(test_batch).data.max(1, keepdim=True)[1]
# if predict[0][0] == 0:
#     print(test_text,"is Bad Mean...")
# else:
#     print(test_text,"is Good Mean!!")

In [72]:
model.save()

AttributeError: ignored

In [75]:
filename = 'finalized_model_10.sav'
joblib.dump(model, filename)

['finalized_model_10.sav']

In [0]:
import joblib